In [ ]:
from adc import align, count
from functools import partial
from importlib.metadata import version
import urllib3
import shutil
import os

import matplotlib.pyplot as plt
import yaml

import tifffile as tf
from scipy.ndimage import gaussian_filter

In [ ]:
__version__ = version('anchor-droplet-chip')
__version__

In [ ]:

def load_data(url, filename):

    if not os.path.exists(filename):
        print(f'loading {filename}')
        c = urllib3.PoolManager()

        with c.request('GET',url, preload_content=False) as resp, open(filename, 'wb') as out_file:
            shutil.copyfileobj(resp, out_file)

        resp.release_conn()
    else:
        print(f'{filename} already exists')
    print(f'reading from disk {filename}')
    return tf.imread(filename)


def process_urls(key, config):
    if 'url' in config[key]:
        return load_data(**config[key])
    else:
        return config[key]

def load_dataset(path):
    with open(path,'r') as f:
        config  = yaml.load(f, Loader=yaml.SafeLoader)
        print (config)

    return {key: process_urls(key, config) for key in config}
     

In [ ]:
data_0h = load_dataset('test_data_0h.yaml')
data_24h = load_dataset('test_data_24h.yaml')


In [ ]:
aligned_stack_0h, tvec_0h = align.align_stack(**data_0h)
aligned_stack_24h, tvec_0h = align.align_stack(**data_24h)
